In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from scipy import ndimage
import skvideo.io  
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import h5py
import pdb
import pandas as pd
import datetime
import scipy as sp
from sklearn.model_selection import KFold
import torch.nn as nn
import torch.nn.functional as F
from matplotlib.patches import Ellipse
from skimage.transform import rescale, resize
from skimage import io
import matplotlib.image as mpimg
import imageio
%matplotlib inline
#plt.style.use(['dark_background'])
#plt.style.use('ggplot')

pos_list = np.concatenate((np.arange(0, num_input_layers), np.arange(num_input_layers, len(evaled), 2)))
for pos in pos_list:
   
    if pos != len(evaled):
        activation_fn = tanh_activation
    else:
        activation_fn = sigmoid_activation
   
    if pos >= num_input_layers and pos < (len(evaled) - 1):
        bias = evaled[pos+1][0,:]
    else:
        bias = np.array(0)
       
    grouped.append({'weights': evaled[pos],
                   'bias': bias,
                    'activation': activation_fn
                   })

In [46]:
class Net(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(in_features=22, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=64)
        self.fc3 = nn.Linear(in_features=64, out_features=32)
        self.fc4 = nn.Linear(in_features = 32, out_features = 3)
    
        
    def forward(self,x):
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = F.tanh(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))
        
        return(x)
    
    def regularizer(self, reg):
        return (self.fc1.weight.pow(2).mean() + self.fc2.weight.pow(2).mean() 
                + self.fc3.weight.pow(2).mean() + self.fc4.weight.pow(2).mean())*reg
    

In [47]:
net = Net()

In [118]:
weights[x_layer, :][:,np.newaxis].shape

(3, 1)

In [120]:
weights[x_layer, :][:,np.newaxis]

(3, 1)

In [134]:
# PARAMETERS
x_layer = 0
y_layer = 1
r_layer = 2
num_eeg_layers = 8
eeg_layers = range(r_layer, r_layer+num_eeg_layers)

# ALLOCATION
grouped = []

for idx, layer in enumerate(net.named_parameters()):
    if idx==0:
        weights = layer[1].detach().numpy().transpose()
        grouped.append({'weights': weights[r_layer+4:,:], 
                       'bias':  np.array(0),
                       'activation': 'tanh'})
        grouped.append({'weights': weights[x_layer, :][np.newaxis,:], 
                       'bias': np.array(0),
                       'activation': 'tanh'})
        grouped.append({'weights': weights[y_layer, :][np.newaxis,:], 
                       'bias':  np.array(0),
                       'activation': 'tanh'})
        grouped.append({'weights': weights[r_layer,:][np.newaxis,:], 
                       'bias':  np.array(0),
                       'activation': 'tanh'})
        for i in np.arange(3):
            grouped.append({'weights': weights[r_layer+i, :][np.newaxis,:], 
                           'bias':  np.array(0),
                           'activation': 'tanh'})  
        grouped.append({'weights': np.zeros((128))[np.newaxis,:], 
                       'bias':  np.array(0),
                       'activation': 'tanh'})
    elif idx==1:
        continue
    elif idx%2==0:
        weights = layer[1].detach().numpy().transpose()
    else:
        # extract biases
        bias = layer[1].detach().numpy().transpose()
        grouped.append({'weights': weights, 
                       'bias': bias,
                       'activation': 'tanh'})
        

In [139]:
[g['weights'] for g in grouped]

[array([[ 0.16058761, -0.01704371,  0.14350045, ...,  0.18762338,
          0.11472836,  0.04913878],
        [-0.06806269, -0.15975949, -0.12928477, ...,  0.01630858,
          0.15842792, -0.03622635],
        [-0.07533103, -0.08673872,  0.01447403, ..., -0.1847184 ,
         -0.05554543, -0.09319096],
        ...,
        [ 0.19646597,  0.12102246,  0.0668177 , ..., -0.08500236,
          0.07795325,  0.10061139],
        [-0.20433518, -0.07033828, -0.10883947, ...,  0.04524916,
         -0.14180276,  0.12116495],
        [ 0.18547279,  0.17648509, -0.11309349, ..., -0.10724597,
         -0.11442567, -0.07986189]], dtype=float32),
 array([[ 0.00757369, -0.18244857, -0.07941167, -0.18207206,  0.01992977,
          0.17256853, -0.09249272, -0.11048423,  0.21000168,  0.19468886,
          0.12124866,  0.0490678 , -0.06870361, -0.0114155 , -0.21029957,
         -0.03048784, -0.01367682, -0.05094144,  0.10760146, -0.09103278,
          0.03538482, -0.17756523, -0.18668352,  0.15374961,  

In [136]:
np.save('random_weights', grouped, allow_pickle=True, fix_imports=False)